In [23]:
import numpy as np
import pandas as pd
import xarray as xr
import h5py
import properscoring as ps

In [8]:
#Load indices
idx = {"u10":0, "v10":1, "t2m":2, "t850":3, "z500":4}
mean = np.load("../../data/stats_v0/global_means.npy").squeeze()
std = np.load("../../data/stats_v0/global_stds.npy").squeeze()

In [9]:
# Open dataset
path_test = "../../data/predictions/ensemble_2022.h5"
test = xr.open_dataset(path_test)

y_test = test.ground_truth
x_single = test.predictions.isel(phony_dim_5 = 0)
x_ens = test.predictions.isel(phony_dim_5 = slice(1,51))

In [30]:
n_var = len(idx) + 2 # Windspeed and precip
length = y_test.shape[1]

In [31]:
# Slicing of Era5 data
variables = np.array([0,1,2,5,14])
time_slice = np.arange(0,(365*4),4)
time_slice_gap = np.arange(0,(366*4),4)

#Use old slices for beginning
lat_subset = slice(60,220)
lon_subset = np.append(np.arange(1390, 1440), np.arange(0, 170))

lat_range = lat_subset.stop - lat_subset.start
lon_range= len(lon_subset)#img_y_subset.stop - img_y_subset.start

# Create h5 file

In [32]:
filename = "../../data/results/crps_metrics.h5"
f = h5py.File(filename, 'a')

try:
    data = f.create_dataset(name = "climatology", shape = (n_var, lat_range, lon_range), dtype = np.float32)
except:
    del f["climatology"]
    data = f.create_dataset(name = "climatology", shape = (n_var, lat_range, lon_range), dtype = np.float32)
f.close()

# Era 5 data

In [17]:
#Load all data
era5_2018 = xr.open_dataset("../../data/out_of_sample/2018.h5").fields.isel(phony_dim_0 = time_slice, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)
era5_2019 = xr.open_dataset("../../data/out_of_sample/2019.h5").fields.isel(phony_dim_0 = time_slice, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)
era5_2020 = xr.open_dataset("../../data/out_of_sample/2020.h5").fields.isel(phony_dim_0 = time_slice_gap, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)
era5_2021 = xr.open_dataset("../../data/out_of_sample/2021.h5").fields.isel(phony_dim_0 = time_slice, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)

#Merge
climatology_data = xr.concat([era5_2018, era5_2019, era5_2020, era5_2021], dim = "phony_dim_0")

#Standardize
for j, var in enumerate(variables):
    climatology_data[:,j] = (climatology_data[:,j] - mean[var]) / std[var]

In [21]:
#Define window
window = 15
n_pred = y_test.shape[0]
n_obs = climatology_data.shape[0]

In [34]:
#Create output
clima_crps = np.zeros(shape = ((n_var-1), lat_range, lon_range))
#Generate slices
for i in range(n_pred):    
    window_slice = np.arange(-window, window+1) + i
    clima_slice = np.append(window_slice, [(window_slice + 365), (window_slice + 2 * 365), (window_slice + 3*365+1)])
    clima_slice = clima_slice[(clima_slice >= 0) & (clima_slice < n_obs)]
    #CRPS for normal variables
    truth = y_test.isel(phony_dim_0 = i, phony_dim_1 = 0)
    pred = climatology_data.isel(phony_dim_0 = clima_slice).transpose(..., "phony_dim_0")
    crps = ps.crps_ensemble(truth, pred)
        
    #CRPS for wind speed
    true_wind = np.sqrt(np.power(truth.isel(phony_dim_2 = 0),2) + np.power(truth.isel(phony_dim_2 = 1),2))
    pred_wind = np.sqrt(np.power(pred.isel(phony_dim_1 = 0),2) + np.power(pred.isel(phony_dim_1 = 1),2))
    crps_wind = ps.crps_ensemble(true_wind, pred_wind)
    clima_crps += np.append(crps, np.expand_dims(crps_wind, axis = 0), axis = 0)
clima_crps = clima_crps / n_pred

data[0:6,...] = clima_crps

ValueError: operands could not be broadcast together with shapes (6,160,220) (6,120,130) (6,160,220) 

# Precip data

In [ ]:
#Load all data
tp_2018 = xr.open_dataset("../../data/precip/out_of_sample/2018.h5").fields.isel(phony_dim_0 = time_slice, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)
tp_2019 = xr.open_dataset("../../data/precip/out_of_sample/2019.h5").fields.isel(phony_dim_0 = time_slice, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)
tp_2020 = xr.open_dataset("../../data/precip/out_of_sample/2020.nc").fields.isel(phony_dim_0 = time_slice_gap, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)
tp_2021 = xr.open_dataset("../../data/precip/out_of_sample/2021.nc").fields.isel(phony_dim_0 = time_slice, phony_dim_1 = variables, phony_dim_2 = lat_slice, phony_dim_3 = lon_slice)

#Merge
climatology_data = xr.concat([tp_2018, tp_2019, tp_2020, tp_2021], dim = "phony_dim_0")

In [ ]:
#Create output
clima_crps = np.zeros(shape = (1, lat_range, lon_range))
#Generate slices
for i in range(n_pred):    
    window_slice = np.arange(-window, window+1) + i
    clima_slice = np.append(window_slice, [(window_slice + 365), (window_slice + 2 * 365), (window_slice + 3*365+1)])
    clima_slice = clima_slice[(clima_slice >= 0) & (clima_slice < n_obs)]
    #CRPS for normal variables
    truth = y_test.isel(phony_dim_0 = i, phony_dim_1 = 0)
    pred = climatology_data.isel(phony_dim_0 = clima_slice).transpose(..., "phony_dim_0")
    crps = ps.crps_ensemble(truth, pred)
        
    #CRPS for wind speed
    true_wind = np.sqrt(np.power(truth.isel(phony_dim_2 = 0),2) + np.power(truth.isel(phony_dim_2 = 1),2))
    pred_wind = np.sqrt(np.power(pred.isel(phony_dim_1 = 0),2) + np.power(pred.isel(phony_dim_1 = 1),2))
    crps_wind = ps.crps_ensemble(true_wind, pred_wind)
    clima_crps += np.append(crps, np.expand_dims(crps_wind, axis = 0), axis = 0)
clima_crps = clima_crps / n_pred

data[6,...] = clima_crps

In [39]:
f.close()